In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
# Load trained model, scaler, onehotencoder
model = load_model('model.h5')

# Load Scaler and Onehotencoder
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('onehot_encoder_geo.pkl', 'rb') as file:
    label_encoder_geo = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)


In [3]:
# Example input data
input_data = {
    'CreditScore': 100,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 80,
    'Tenure': 1,
    'Balance': 6000,
    'NumOfProducts': 10,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 10000
}

In [4]:
# One-hot encoding geography
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Python\Gen AI Course\ANN Project\myenv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
[[input_data['Geography']]]

[['France']]

In [36]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,100,France,Male,80,1,6000,10,1,1,10000


In [37]:
# Encode Categorical value
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,100,France,1,80,1,6000,10,1,1,10000


In [38]:
# Concatinate input_df with one hot encoded columns
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,100,1,80,1,6000,10,1,1,10000,1.0,0.0,0.0


In [39]:
# Scale the input data
input_scaled = scaler.transform(input_df)

In [40]:
input_scaled

array([[-5.72485068,  0.91324755,  3.90769104, -1.38944225, -1.12240462,
        14.60942677,  0.64920267,  0.97481699, -1.57229902,  1.00150113,
        -0.57946723, -0.57638802]])

In [41]:
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


array([[1.]], dtype=float32)

In [42]:
prediction_proba = prediction[0][0]
prediction_proba

np.float32(1.0)

In [43]:
if prediction_proba > 0.5:
    print('Customer is likely to churn')
else:
    print('Customer is not likely to churn')

Customer is likely to churn


In [44]:
label_encoder_gender.classes_

array(['Female', 'Male'], dtype=object)

In [45]:
label_encoder_geo.categories_

[array(['France', 'Germany', 'Spain'], dtype=object)]